### 1) Instalar mlflow 2.x

In [1]:
#!pip install mlflow==2.2.1

### 2) Iniciar servidor de mlflow (solo necesario para trabajar de forma local)
Debido a que mientras el servidor esté corriendo la línea de código va a seguir corriendo

Instrucciones:
- Abrir una consola. Por ejemplo Anaconda Prompt

- Pararse en el root del repo

- Correr:

    
    `mlflow server --default-artifact-root <ruta para almacenar artefactos>`
    

- Correr v2: Por defecto, si no se indica una ruta se crea la carpeta "mlruns"
    `mlflow server`


- Este comando iniciará el servidor de MLflow en tu máquina local y estará listo para recibir experimentos y modelos. Además, puedes acceder a la interfaz de usuario de MLflow ingresando en tu navegador la dirección http://localhost:5000.

In [2]:
#######  mlflow server --default-artifact-root <ruta para almacenar artefactos>


#######  mlflow server

### 3) Import mlflow

In [4]:
import mlflow


import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xg

### 4) Conectarse a mlflow
Se puede utilizar mlflow:

- De forma local

- mlflow en un cluster kubernetes + cloud sql

- mlflow en GCS (UI en cloud run). En este ejemplo se va a utilizar este método

In [5]:
#conectar a mlflow
path_mlflow_cloudrun = 'url-mlflow-cloud-run' #------ complete with the mlflow url

mlflow.set_tracking_uri(path_mlflow_cloudrun)

### 5) Setear experimento.
Si el experimento no existe, se crea automáticamente

In [6]:
experiment_name = 'test_mlflow_cloudrun'
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='gs://mlflow_cloudrun_tracking_uri/861524145491099593', creation_time=1678333077766, experiment_id='861524145491099593', last_update_time=1678333077766, lifecycle_stage='active', name='test_mlflow_cloudrun', tags={}>

In [ ]:
mlflow.get_experiment_by_name(experiment_name)

### 6) Generación de parámetros, variables, artifacts a guardar
- Todos estos artefactos son generados durante el entrenamiento del modelo

- En este ejemplo se crean algunos para indicarlos como ejemplos

### 6.0 Entrenar modelo (generar datos y entrenar modelo)

In [8]:
#### generar datos ###

# parametros
len_data = 1000
number_columns = 6
data = []
list_variables = ["240FY050.RO02" , "SGM-PI9514", "SSTRIPPING015", "SGM-PI9516" , "SGM-PI9512", "target"]


# seed - replicar resultados
np.random.seed(42)


# generar data aleatoria
for column in range(number_columns):
    random_choise = np.random.choice(10) + 1 # amplitud
    data_column = np.random.rand(len_data)
    data_column = random_choise * data_column
    data.append(data_column)
    
# a dataframe
data = pd.DataFrame(data).T
data.columns = list_variables

# serparar train y test
features = list(set(list_variables) - set(['target']))
X_train, X_test, y_train, y_test = train_test_split(data[features], data['target'], test_size = 0.2, random_state=42)

print('TRAIN')
print('X_train', X_train.shape)
print('y_train', y_train.shape)

print('\nTEST')
print('X_test', X_test.shape)
print('y_test', y_test.shape)

TRAIN
X_train (800, 5)
y_train (800,)

TEST
X_test (200, 5)
y_test (200,)


In [9]:
#### entrenar modelo ####

model = LinearRegression()
#model = RandomForestRegressor(random_state = 42)
#model = xg.XGBRegressor(objective ='reg:linear', n_estimators = 10, seed = 42)

model.fit(X_train, y_train)



#### predicción y evaluación ####

### RMSE
rmse_train = mean_squared_error(y_train, 
                                model.predict(X_train),
                                squared = False)

rmse_test = mean_squared_error(y_test, 
                               model.predict(X_test),
                               squared = False)


### R2
r2_score


r2_train = r2_score(y_train,
                   model.predict(X_train))

r2_test = r2_score(y_test,
                   model.predict(X_test))

### 6.1) Generar parámetros

In [10]:
# print de listado de tags
print('listado de tags', list_variables)

listado de tags ['240FY050.RO02', 'SGM-PI9514', 'SSTRIPPING015', 'SGM-PI9516', 'SGM-PI9512', 'target']


In [11]:
# tipo modelo
model_type = "lr"
model_type

'lr'

In [12]:
# fechas de los datos
start_train = "2020-01-01"
end_train = "2022-12-01"

### 6.2) Generar métricas

In [13]:
# print de las métricas
print('rmse_train: ', rmse_train)
print('rmse_test: ', rmse_test)
print('r2_train: ', r2_train)
print('r2_test: ', r2_test)

rmse_train:  1.4387312960740368
rmse_test:  1.4632467048976407
r2_train:  0.007202043897398536
r2_test:  -0.011984948488191094


### 6.3) Generar artefactos
- Modelos
- Data
- Gráficos
- ETC

In [14]:
# generar artefacto pickle con el modelo y borrar del local
model_name = 'model.pkl'
with open(model_name, 'wb') as file:
    pickle.dump(model, file)

In [15]:
# generar artefacto data csv y borrar del local
data_name = 'data.csv'
data.to_csv(data_name)

### 7. Guardar resultados del RUN
- Se tiene un "EXPERIMENT" el cual es la unidad "grande" que está conformada por unidades más pequeñas


- Cuando uno corre el entrenamiento de un modelo (que podría ser llamado un experimento), en mlflow recibe el nombre de "RUN"


- Por lo tanto un "EXPERIMENT" está conformado múltiples "RUNS"

In [16]:
#iniciar run
run_name = model_type
mlflow.start_run(run_name = run_name)
run = mlflow.active_run()

In [17]:
run

<ActiveRun: >

In [18]:
# guardar parámetros. ("Nombre Parámetro", "Valor")
mlflow.log_param("Tags", str(list_variables))
mlflow.log_param("Modelo", model_type)
mlflow.log_param("Inicio Train", start_train)
mlflow.log_param("Fin Train", end_train)

'2022-12-01'

In [19]:
# guardar variables ("Nombre", Valor)
mlflow.log_metric("RMSE_train", rmse_train)
mlflow.log_metric("RMSE_test", rmse_test)
mlflow.log_metric("R2_train", r2_train)
mlflow.log_metric("R2_test", r2_test)

In [20]:
# guardar artefactos. Forma simple es que estén guardados en el local y luego subirse a mlflow

# guardar modelo
mlflow.log_artifact(model_name)
os.remove(model_name)

# guardar data
mlflow.log_artifact(data_name)
os.remove(data_name)

### 8. Terminar RUN

In [21]:
# terminar run
mlflow.end_run()

### 9. EXTRAS
Mlflow permite:

- Guardar el artefacto del modelo en una carpeta "especial" para luego hacer el deploy de este utilizando servicios de mlflow (mlflow corriendo en un cluster en la nube)

- De todos los RUN elegir el que tiene mejor métrica

- El modelo eleguido hacer el deploy


Actualmente se utiliza otras formas de hacer el deploy de los modelos y mlflow solo se utilizará para tener registro de los diferentes experimentos realizados